<a href="https://colab.research.google.com/github/edhillon7/Neural-Networks/blob/main/Campus_Assignment_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading the labeled and unlabeled data

In [18]:
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/drive/MyDrive/Copy of HV-AI-2025.zip" /content/HV-AI-2025.zip
!unzip HV-AI-2025.zip
!rm -rf /content/__MACOSX
!mv /content/HV-AI-2025/* /content/
!rm -rf /content/HV-AI-2025
!rm /content/HV-AI-2025.zip
!rm -rf /content/sample_data
from google.colab import output

Streaming output truncated to the last 5000 lines.
  inflating: HV-AI-2025/unlabeled_data/images/OIP-Pn9quPZZ4H8fJ939NLrsIgHaFj.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/OIP-IIeVvMzmadmWndWvfSpaKQHaE9.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/OIP-4t4U7LOzS-GByUxY_8ylMwHaFf.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/OIP-6Te7FgW0INPUUNl32tgW1QHaHa.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/921.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/OIP-7z-PsChIDdT8KECFqWQCRwHaEr.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/OIP-CE_0FI0DXtkH3rmQt4OiyAHaE7.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/OIP-RsoOv56vjNaHJcERlx21SwHaHv.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/ea37b80e2ef5023ed1584d05fb1d4e9fe777ead218ac104497f5c97faee9bdba_640.jpg  
  inflating: HV-AI-2025/unlabeled_data/images/OIP-LiaYVqnIYT9kpq_TwYmKJQHaF_.jpeg  
  inflating: HV-AI-2025/unlabeled_data/images/OIP-njs-RicCRdYTzNrlo4MoawHaH6.jpeg  
  inflating: HV

#Downloading the test data

In [10]:
!gdown https://drive.google.com/uc?id=1aszVlQFQOwJTy9tt79s7x87VJyYw-Sxy
!unzip HV-AI-2025-Test.zip
!rm -rf /content/__MACOSX
!mv /content/HV-AI-2025-Test/* /content/
!rm -rf /content/HV-AI-2025-Test
!rm /content/HV-AI-2025-Test.zip
!rm -rf /content/sample_data
from google.colab import output
output.clear()

# **Load/Preprocess **data****

In [19]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# ✅ Correct path to labels.csv
labels_df = pd.read_csv('labeled_data/labeled_data.csv')

labels_df['path'] = labels_df['img_name'].apply(lambda x: os.path.join('labeled_data/images', x))
unique_labels = labels_df['label'].unique()
label2idx = {label: idx for idx, label in enumerate(unique_labels)}
print(label2idx)

# Split into train and val
train_df, val_df = train_test_split(
    labels_df,
    test_size=0.2,
    stratify=labels_df['label'],
    random_state=42
)
unique_labels = labels_df['label'].unique()
print(unique_labels)





{'cane': 0, 'cavallo': 1, 'elefante': 2, 'farfalla': 3, 'gallina': 4, 'gatto': 5, 'mucca': 6, 'pecora': 7, 'ragno': 8, 'scoiattolo': 9}
['cane' 'cavallo' 'elefante' 'farfalla' 'gallina' 'gatto' 'mucca' 'pecora'
 'ragno' 'scoiattolo']


In [16]:
!ls /content

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx,'path']).convert('RGB')
        label_str = self.df.loc[idx, 'label']
        label = label2idx[label_str]
        if self.transform: img = self.transform(img)
        return img, label

train_ds = ImageDataset(train_df, train_transform)
val_ds = ImageDataset(val_df, val_transform)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)


# **Model initialization/Training**

In [21]:
import torchvision.models as models
import torch.nn as nn

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 10 classes


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 140MB/s]


In [22]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

best_acc = 0
for epoch in range(10):  # adjust based on time
    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    # validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct/total
    print(f"Epoch {epoch}: Val Acc = {acc:.4f}")
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_phase1.pth")


Epoch 0: Val Acc = 0.4808
Epoch 1: Val Acc = 0.4872
Epoch 2: Val Acc = 0.6667
Epoch 3: Val Acc = 0.4359
Epoch 4: Val Acc = 0.5321
Epoch 5: Val Acc = 0.5577
Epoch 6: Val Acc = 0.5705
Epoch 7: Val Acc = 0.5962
Epoch 8: Val Acc = 0.5897
Epoch 9: Val Acc = 0.5641


# **Model Inference**

In [24]:
!unzip "/content/drive/MyDrive/Copy of HV-AI-2025-Test.zip" -d /content/test_data


Streaming output truncated to the last 5000 lines.
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-k-v3LAZqBmumWMR26RPORQHaEK.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-YzZWlCiYdyG6V0W0BZyZQAHaLJ.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-ADJJeiBee8FJRpVqBoYOAAHaE8.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-rezzMoaidVH_SiLTrKg21wHaGU.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-pibNhK8eu_PuwTuJzA6SngHaFj.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-aNtiDa__wQpbA5Saz4x4VAHaE8.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-8PmnmK7F_vrfXwdo1ERnxAHaFj.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-HJH1KvHRZ5-CtV2QQX4urwHaGU.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/OIP-Kga1Rhhp0w_oNB0fhZrVFAAAAA.jpeg  
  inflating: /content/test_data/HV-AI-2025-Test/test_images/ea37b00

In [37]:

import os
import pandas as pd

test_dir = "/content/test_data/HV-AI-2025-Test/test_images"
test_images = sorted(os.listdir(test_dir))
test_df = pd.DataFrame({'img_name': test_images})
test_df['path'] = test_df['img_name'].apply(lambda x: os.path.join(test_dir, x))
print(test_df.head())





      img_name                                               path
0  0_1034.jpeg  /content/test_data/HV-AI-2025-Test/test_images...
1  0_1045.jpeg  /content/test_data/HV-AI-2025-Test/test_images...
2  0_1085.jpeg  /content/test_data/HV-AI-2025-Test/test_images...
3   0_110.jpeg  /content/test_data/HV-AI-2025-Test/test_images...
4  0_1104.jpeg  /content/test_data/HV-AI-2025-Test/test_images...


In [38]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx, 'path']).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.df.loc[idx, 'img_name']

test_ds = TestDataset(test_df, test_transform)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2)


In [39]:
import torch
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recreate the model
model = models.resnet18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, len(label2idx))  # output layer

# Load weights
model.load_state_dict(torch.load("best_phase1.pth", map_location=device))
model.to(device)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [40]:
idx2label = {v: k for k, v in label2idx.items()}
print(idx2label)


{0: 'cane', 1: 'cavallo', 2: 'elefante', 3: 'farfalla', 4: 'gallina', 5: 'gatto', 6: 'mucca', 7: 'pecora', 8: 'ragno', 9: 'scoiattolo'}


In [41]:
all_preds = []

with torch.no_grad():
    for imgs, img_names in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().numpy()
        for name, pred_idx in zip(img_names, preds):
            all_preds.append([name, idx2label[pred_idx]])


In [42]:
pred_df = pd.DataFrame(all_preds, columns=['path', 'predicted_label'])
pred_df.to_csv('phase1_predictions.csv', index=False)
print(pred_df.head())


          path predicted_label
0  0_1034.jpeg           gatto
1  0_1045.jpeg           gatto
2  0_1085.jpeg           gatto
3   0_110.jpeg            cane
4  0_1104.jpeg           gatto


# **Helper Functions**

In [ ]:
import requests

def send_results_for_evaluation(name, csv_file, email):
    url = "http://43.205.49.236:5050/inference"
    files = {'file': open(csv_file, 'rb')}
    data = {'email': email, 'name':name}
    response = requests.post(url, files=files, data=data)
    return response.json()



# ***Test Inference***


This function is used to save the csv file and send it to the evaluation server.

Format of CSV file (Follow the header names strictly):

          path (str)               predicted_label(str)
    test_data/images/xx.jpg               class_1         
    test_data/images/yy.jpg               class_2         
             :                               :                          
             :                               :                          

Once the prediction file is saved as shown in the above format, you can send it to the evaluation server along with your email.

Caution: check your **email** before executing the cell.


In [ ]:

print('Accuracy: ')
print(send_results_for_evaluation('name', 'path to pred.csv', 'your_email'))


